Example Try Finally remplacé par with

In [1]:
file = open("./fichier.txt")
try:
    for line in file:
        print(line.strip())
finally:
    file.close()

Ligne 1
Ligne 2
Ligne 3
Ligne 4
Ligne 5


Remplacé par with

In [3]:
with open("./fichier.txt") as file:
    for line in file:
        print(line.strip())


Ligne 1
Ligne 2
Ligne 3
Ligne 4
Ligne 5


Faire son propre context manager pour l'utlisation avec with

In [13]:
class ContexteIllustration:
    def __enter__(self):
        print(" Je suis entrée dans le contexte, des opération de préparations")
        
    def __exit__(self, exc_type, exc_value, traceback):
        print("On quitte le contexte")
        if exc_type is None:
            print("Pas d'erreurs pendant les instructions dans le contexte de with")
            return True
        else:
            print(f"Type of the Exception : {exc_type}")
            print(f"Value of the Isntance : {exc_value}")
            return True


with ContexteIllustration():
    print( " Je suis dans le contexte de with : Instruction 1")
    print( " Je suis dans le contexte de with : Instruction 2")
    raise ValueError(" La donnée est pas bonne")
# d'exception         

 Je suis entrée dans le contexte, des opération de préparations
 Je suis dans le contexte de with : Instruction 1
 Je suis dans le contexte de with : Instruction 2
On quitte le contexte
Type of the Exception : <class 'ValueError'>
Value of the Isntance :  La donnée est pas bonne


Exemple avec application de patch temporaire

In [14]:

class Config:
    etat = "normal"

config = Config()

etat_orignal = config.etat

try: 
    config.etat = "temporaire"
    print("Instruction 1")
finally:
    config.etat = etat_orignal

Instruction 1


In [18]:
class ConfigManager:
    def __init__(self, currentConfig, etatTemporaire):
        self.currentConfig = currentConfig
        self.etatTemporaire = etatTemporaire
        
    def __enter__(self):
        self.currentEtat = self.currentConfig.etat
        self.currentConfig.etat = self.etatTemporaire
        print(f"Etat Courant : {self.currentConfig.etat}")
        print("fin __enter__")
        
    def __exit__(self, exc_type, exc_value, traceback):
        self.currentConfig.etat = self.currentEtat
        print(f"Etat Courant : {self.currentConfig.etat}")
        print("fin __exit__")
        return True
        
config = Config()

with ConfigManager(config, "temp"):
    print("instruction 1")
config.etat = "Precision10-5"
with ConfigManager(config, "temporatire_etat"):
    print("instruction 2")




Etat Courant : temp
fin __enter__
instruction 1
Etat Courant : normal
fin __exit__
Etat Courant : temporatire_etat
fin __enter__
instruction 2
Etat Courant : Precision10-5
fin __exit__


In [20]:
class ContexteIllustrationWithRetour:
    def __enter__(self):
        print(" Je suis entrée dans le contexte, des opération de préparations")
        return "Ceci est ma valeur de retour"
        
    def __exit__(self, exc_type, exc_value, traceback):
        print("On quitte le contexte")
        if exc_type is None:
            print("Pas d'erreurs pendant les instructions dans le contexte de with")
            return True
        else:
            print(f"Type of the Exception : {exc_type}")
            print(f"Value of the Isntance : {exc_value}")
            return True


with ContexteIllustrationWithRetour() as valeur_de_retour_de_enter:
    print(valeur_de_retour_de_enter)
    print( " Je suis dans le contexte de with : Instruction 1")
    print( " Je suis dans le contexte de with : Instruction 2")
# d'exception       

 Je suis entrée dans le contexte, des opération de préparations
Ceci est ma valeur de retour
 Je suis dans le contexte de with : Instruction 1
 Je suis dans le contexte de with : Instruction 2
On quitte le contexte
Pas d'erreurs pendant les instructions dans le contexte de with
